## Continuación clase anterior
- Volveremos a montar el último deploy realizado junto con un pod de Ubuntu fuera de la red local del deployment:
    - Iniciar Docker
    - Crear el clúster (`minikube start -drive docker`)
    - Desplegar el deployment (`load-balancer.yaml`)
    - Desplegar el pod de ubuntu (`ubuntu.yaml`)
    - Actualizar las dependencias internas de `Ubuntu`
        ```
        kubectl exec -it ubuntu -- bash
        apt-get update
        apt-get upgrade
        apt install curl
        curl http://hello:8080
       ```

## Trabajando con imágenes locales
Hasta ahora hemos realizado deployments con imágenes que estan en el DockerHub. Pero, Qué pasa si quiero utilizar una imagen local? O una imagen en un repositorio (registro) en mi nube?

Para hacerlo de manera local, se debe conectar el registro de imagenes localo, con el registro de imagenes de `minikube`. Para eso, se debe correr el siguiente comando:
```
eval $(minikube -p minikube docker-env)
```

Windows
```
REM @FOR /f "tokens=*" %i IN ('minikube -p minikube docker-env --shell cmd') DO @%i
```

Y seguidamente, construir la imagenes (o imágenes) que se quiere pasar al registro de `minikube`. Nota, esto se debe hacer cada vez qque se quiera volver a ejecutar.
___

Vamos a usar la API del modelo `iris` que hemos venido usando...

```
imagePullPolicy: Never
```

    - Modificar el deployment para cambiar el label de la red interna (y el nombre del deployment y pods)

    - Agregar el readiness y el liviness (Qué era esto?)

    - Jugar con los puertos

# Cloud Computing: Azure

- Vamos a crear una cuenta gratuita para estudiantes en Azure:
    - [Azure](https://azure.microsoft.com/en-us/free/students/)
- Instalar la línea de comando (CLI) de Azure: [Cómo instalar](https://learn.microsoft.com/es-es/cli/azure/install-azure-cli-macos)
- Una vez instalado, vamos a registrarnos a través de una línea de comandos:
    - `az login`
- Vamos a crear un registro de contenedores y subir (pull) una imagen (la de la api de iris)
- Vamos a crear un clúster de kubernetes
- Vamos a conectar el clúster de kubernetes con nuestra cli local (kubeconfig)
    - `az aks get-credentials --resource-group <group name> --name <cluster name> --admin`
- Vamos a asociar el registro de contenedores con el clúster de kubernetes
    ```
    kubectl create secret docker-registry <secret-name> \
        --docker-server=<container-registry-name>.azurecr.io \
        --docker-username=<service-principal-ID> \
        --docker-password=<service-principal-password>
    ```
- Vamos a crear un manifiesto similar al que ya tenemos, agregando lo siguiente:
```
apiVersion: v1
kind: Service
metadata:
  name: load-balancer
spec:
  type: LoadBalancer
  ports:
  - port: 5000
    targetPort: 8000
  selector:
    role: vpn-api
```

```
  containers:
  - name: container-iris
    image: registrycristian.azurecr.io/iris:latest
    imagePullPolicy: IfNotPresent
    ports:
    - containerPort: 8000
    readinessProbe:
      httpGet:
        path: /
        port: 8000
      initialDelaySeconds: 5
      periodSeconds: 10
    livenessProbe:
      tcpSocket:
        port: 8000
      initialDelaySeconds: 15
      periodSeconds: 20
  imagePullSecrets:
    - name: secreto
```

```
kubectl create secret docker-registry <secret-name> \
    --namespace <namespace> \
    --docker-server=<container-registry-name>.azurecr.io \
    --docker-username=<service-principal-ID> \
    --docker-password=<service-principal-password>
```